In [28]:
import numpy.random as nr
from prettytable import PrettyTable

In [29]:
class UniformGenerator:
    def __init__(self, a, b):
        if not 0 <= a <= b:
            raise ValueError('Параметры должны удовлетворять условию 0 <= a <= b')
        self._a = a
        self._b = b

    def next(self):
        return nr.uniform(self._a, self._b)

In [30]:
class PoissonGenerator:
    def __init__(self, la):
        self._la = la
    
    def next(self):
        return nr.poisson(self._la)

In [31]:
class Generator:
    def __init__(self, generator):
        self._generator = generator
        self._receivers = set()

    def add_receiver(self, receiver):
        self._receivers.add(receiver)

    def remove_receiver(self, receiver):
        try:
            self._receivers.remove(receiver)
        except KeyError:
            pass
    
    def next_time(self):
        return self._generator.next()
    
    def emit_request(self):
        for rec in self._receivers:
            rec.receive_request()

In [32]:
class Queue:
    def __init__(self):
        self._current_queue_size = 0
        self._max_queue_size = 0
        
    @property
    def max_queue_size(self):
        return self._max_queue_size

    @property
    def current_queue_size(self):
        return self._current_queue_size
    
    def add(self):
        self._current_queue_size += 1
        
    def remove(self):
        self._current_queue_size -= 1
        
    def increase_size(self):
        self._max_queue_size += 1

In [33]:
class Processor(Generator):
    def __init__(self, generator, return_probability):
        super().__init__(generator)
        self._generator = generator
        self._processed_requests = 0
        self._return_probability = return_probability
        self._reentered_requests = 0
        self._queue = Queue()
        
    @property
    def processed_requests(self):
        return self._processed_requests
    
    @property
    def reentered_requests(self):
        return self._reentered_requests
    
    @property
    def queue(self):
        return self._queue
    
    def process(self):
        if self._queue.current_queue_size > 0:
            self._processed_requests += 1
            self._queue.remove()
            self.emit_request()
            if nr.random_sample() < self._return_probability:
                self._reentered_requests += 1
                self.receive_request()

    def receive_request(self):
        self._queue.add()
        if self._queue.current_queue_size > self._queue.max_queue_size:
            self._queue.increase_size()

    def next_time_period(self):
        return self._generator.next()

In [65]:
class Model:
    def __init__(self, a, b, la, ret_prob):
        self._generator = Generator(UniformGenerator(a, b))
        self._processor = Processor(PoissonGenerator(la), ret_prob)
        self._generator.add_receiver(self._processor)
        
    def event_based_modelling(self, request_count):
        generator = self._generator
        processor = self._processor

        gen_period = generator.next_time()
        proc_period = gen_period + processor.next_time()
        while processor.processed_requests < request_count:
#         while proc_period < request_count:
            if gen_period <= proc_period:
                generator.emit_request()
                gen_period += generator.next_time()
            if gen_period >= proc_period:
#                 cur_queue = processor.queue.current_queue_size
                processor.process()
#                 if processor.queue.current_queue_size == cur_queue:
#                     request_count += 1
                if processor.queue.current_queue_size > 0:
                    proc_period += processor.next_time()
                else:
                    proc_period = gen_period + processor.next_time()

        return (processor.processed_requests, processor.reentered_requests,
                processor.queue.max_queue_size, proc_period)

    def time_based_modelling(self, request_count, dt):
        generator = self._generator
        processor = self._processor

        gen_period = generator.next_time()
        proc_period = gen_period + processor.next_time()
        current_time = 0
        while processor.processed_requests < request_count:
#         while current_time < request_count:
            if gen_period <= current_time:
                generator.emit_request()
                gen_period += generator.next_time()
            if current_time >= proc_period:
#                 cur_queue = processor.queue.current_queue_size
                processor.process()
#                 if processor.queue.current_queue_size == cur_queue:
#                     request_count += 1
                if processor.queue.current_queue_size > 0:
                    proc_period += processor.next_time()
                else:
                    proc_period = gen_period + processor.next_time()
            current_time += dt

        return (processor.processed_requests, processor.reentered_requests,
                processor.queue.max_queue_size, current_time)

In [66]:
def print_results(res, rps):
    table = PrettyTable()
    table.add_column("Вероятность возврата", rps)
    table.add_column("Минимальная длина очереди", res)
#     table.add_column("Время", results_t)
    print(table)

In [82]:
a = 0
b = 10
la = 6
rp = 0.1
# modelev1 = Model(a, b, la, 0)
# modeltm1 = Model(a, b, la, 0)
# modelev2 = Model(a, b, la, rp)
# modeltm2 = Model(a, b, la, rp)

rps = [0, 0.05, 0.1, 0.2, 0.5, 0.75, 0.9]
res_t = []
res_e = []
for i in range(len(rps)):
    modelt = Model(a, b, la, rps[i])
    modele = Model(a, b, la, rps[i])
    processedt, reenteredt, queue_sizet, timet = modelt.time_based_modelling(1000, 0.001)
    processede, reenterede, queue_sizee, timee = modele.event_based_modelling(1000)
    res_t.append(queue_sizet)
    res_e.append(queue_sizee)
print("Принцип dt")
print_results(res_t, rps)
print("Событийный принцип")
print_results(res_e, rps)

# print(modelev1.event_based_modelling(1000))
# print(modeltm1.time_based_modelling(1000, 0.001))
# print(modelev2.event_based_modelling(1000))
# print(modeltm2.time_based_modelling(1000, 0.001))

Принцип dt
+----------------------+---------------------------+
| Вероятность возврата | Минимальная длина очереди |
+----------------------+---------------------------+
|          0           |            189            |
|         0.05         |            222            |
|         0.1          |            290            |
|         0.2          |            376            |
|         0.5          |            708            |
|         0.75         |            921            |
|         0.9          |            1086           |
+----------------------+---------------------------+
Событийный принцип
+----------------------+---------------------------+
| Вероятность возврата | Минимальная длина очереди |
+----------------------+---------------------------+
|          0           |            199            |
|         0.05         |            209            |
|         0.1          |            335            |
|         0.2          |            401            |
|         0.5   